In [1]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Specify the path to the dataset file
data_dir = '/content/drive/MyDrive/wikihow_data/'

# Load the WikiHow dataset
dataset = load_dataset('wikihow', 'all', data_dir=data_dir)


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/157252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5599 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5577 [00:00<?, ? examples/s]

In [3]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

#model_checkpoint = "t5-small"
model_checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
#prefix = "summarize: "


def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    labels = tokenizer(text_target=examples["headline"], max_length=1024, truncation=True,padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
small_train_dataset = dataset["train"].select(range(20000))

In [14]:
train_tokenized_dataset = small_train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [15]:
test_tokenized_dataset = dataset["test"].map(preprocess_function, batched=True)
validation_tokenized_dataset = dataset["validation"].map(preprocess_function, batched=True)

In [17]:
small_train_dataset.save_to_disk("/content/drive/MyDrive/wikihow_data/BART_small_train_dataset")
test_tokenized_dataset.save_to_disk("/content/drive/MyDrive/wikihow_data/BART_test_tokenized_dataset")
validation_tokenized_dataset.save_to_disk("/content/drive/MyDrive/wikihow_data/BART_validation_tokenized_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5577 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5599 [00:00<?, ? examples/s]